In [150]:
import numpy as np
import pandas as pd


In [151]:
# 1. 전국도시공원정보 표준데이터 파일 읽기
park = pd.read_csv('data/경기도도시공원정보표준데이터.csv', encoding='euc-kr')
park.head(1)

,관리번호,공원명,공원구분,소재지도로명주소,소재지지번주소,위도,경도,공원면적,공원보유시설(운동시설),공원보유시설(유희시설),공원보유시설(편익시설),공원보유시설(교양시설),공원보유시설(기타시설),지정고시일,관리기관명,전화번호,데이터기준일자,제공기관코드,제공기관명
0,11170-00048,녹색소공원,소공원,NaN,서울특별시 용산구 효창동 287-6,37.539217,126.962694,1569.9,NaN,NaN,NaN,NaN,NaN,2012-10-19,서울특별시 용산구청,02-2199-7605,2022-12-20,3020000,서울특별시 용산구


In [152]:
park[ (park.소재지도로명주소.str.find('경기도')>=0) & (park.소재지지번주소.isna()) ]

,관리번호,공원명,공원구분,소재지도로명주소,소재지지번주소,위도,경도,공원면적,공원보유시설(운동시설),공원보유시설(유희시설),공원보유시설(편익시설),공원보유시설(교양시설),공원보유시설(기타시설),지정고시일,관리기관명,전화번호,데이터기준일자,제공기관코드,제공기관명
365,41360-00078,별내택지어린이공원3,어린이공원,경기도 남양주시 별내동 별내3로 285,NaN,37.662716,127.116471,2743.0,NaN,NaN,NaN,NaN,NaN,NaN,경기도 남양주시청,031-590-8601,2022-12-07,3990000,경기도 남양주시
5426,41360-00161,녹촌제1호소공원,소공원,경기도 남양주시 화도읍 경춘보학1길 7,NaN,37.650624,127.297329,1228.0,NaN,NaN,NaN,NaN,NaN,NaN,경기도 남양주시청,031-590-8601,2022-12-07,3990000,경기도 남양주시
7145,41115-00042,66호소공원,소공원,경기도 수원시 팔달구 팔달로165번길 34-2,NaN,37.283442,127.004854,347.0,NaN,휴게쉼터,NaN,NaN,NaN,NaN,경기도 수원시 화성사업소 문화유산시설과,031-228-4473,2022-06-20,3740000,경기도 수원시
7164,41115-00007,아낌없이주는나무,어린이공원,경기도 수원시 팔달구 세지로 399번길 46,NaN,37.287167,127.027868,3146.7,다목적구장노인정1 +다목적구장(풋살+게이트볼)2,조합놀이대1,노인정1,NaN,NaN,NaN,경기도 수원시 팔달구 녹지공원과,031-228-7043,2022-06-20,3740000,경기도 수원시
7432,41113-00007,권선늘푸른,근린공원,경기도 수원시 권선구 동탄원천로 818,NaN,37.248008,127.040080,33855.6,농구장1,바닥분수1+ 야외무대1,화장실1,도서관,주차장,NaN,경기도 수원시 권선구 녹지공원과,031-228-6195,2022-06-20,3740000,경기도 수원시


In [153]:
len(park[ (park.소재지도로명주소.isna()) & (park.소재지지번주소.str.find('경기도')>=0)])

3470

In [154]:
# 1) 공원명, 소재지지번주소, 공원면저 컬럼만 선택
park = park[['공원명', '소재지지번주소', '공원면적']]
# 2) 경기도만 추출
park = park[park.소재지지번주소.str.find('경기도')>=0]  



In [155]:
park[park.공원명 == '아낌없이주는나무']

,공원명,소재지지번주소,공원면적


In [156]:
# 3) 소재지지번주소에서 시군구 이름 추출
slist = []
for addr in park.소재지지번주소:
    if addr.find('구 ') >=0:
        slist.append(''.join(addr.split()[1:3]))
    else:
        slist.append(addr.split()[1])

park['시군구'] = slist


In [157]:
park.isna().sum()

공원명        0
소재지지번주소    0
공원면적       0
시군구        0
dtype: int64

In [158]:
# 공원면적을 정수형으로 바꿈
park.공원면적 = park.공원면적.astype(int)

In [159]:
park.head()

,공원명,소재지지번주소,공원면적,시군구
212,꿈나무공원,경기도 이천시 중리동 15,2526,이천시
213,남천공원,경기도 이천시 중리동 210,3830,이천시
214,향교공원,경기도 이천시 창전동 408,1757,이천시
215,개나리공원,경기도 이천시 창전동 414-13,1650,이천시
216,달맞이공원,경기도 이천시 창전동 423,1650,이천시


In [160]:
# 4) 시군구 별로 공원면적을 합함
park = park.pivot_table('공원면적', '시군구', aggfunc='sum')
park.head()

,공원면적
시군구,
가평군,511673
고양시덕양구,3287283
고양시일산동구,2722980
고양시일산서구,922486
과천시,315666


In [161]:
# 2. 경기도 주민등록인구집계현화일 읽는다.
except_city = ['경기도 수원시', '경기도 고양시', '경기도 용인시', '경기도 성남시', '경기도 안산시', '경기도 안양시']

pop = pd.read_csv('../02.Pandas/data/주민등록인구집계현황.csv', encoding='euc-kr')

# 1) 행정구역구분명, 행정구역명, 총 인구수 컬럼 선택
pop = pop[['행정구역구분명', '행정구역명', '총 인구수']]
# 2), 3) 시군구 데이터만 추출, 시 데이터 제외
pop = pop[((pop.행정구역구분명 == '시군') | (pop.행정구역구분명 == '구')) & ~(pop['행정구역명'].str.strip().isin(except_city))]

# pop = pop[['행정구역명', '총 인구수']]
pop.head()

,행정구역구분명,행정구역명,총 인구수
1,시군,경기도 가평군,62197
9,구,경기도 고양시 덕양구,487874
31,구,경기도 고양시 일산동구,296590
44,구,경기도 고양시 일산서구,290738
56,시군,경기도 과천시,77775


In [162]:
# 4) 시군구 이름만 추출
pop['시군구'] = pop.행정구역명.apply(lambda x: ''.join(x.split()[1:]))
pop.head()

,행정구역구분명,행정구역명,총 인구수,시군구
1,시군,경기도 가평군,62197,가평군
9,구,경기도 고양시 덕양구,487874,고양시덕양구
31,구,경기도 고양시 일산동구,296590,고양시일산동구
44,구,경기도 고양시 일산서구,290738,고양시일산서구
56,시군,경기도 과천시,77775,과천시


In [163]:
pop = pop[['시군구', '총 인구수']]
pop.head()

,시군구,총 인구수
1,가평군,62197
9,고양시덕양구,487874
31,고양시일산동구,296590
44,고양시일산서구,290738
56,과천시,77775


In [164]:
pop.set_index('시군구', inplace=True)
pop.head()

,총 인구수
시군구,
가평군,62197
고양시덕양구,487874
고양시일산동구,296590
고양시일산서구,290738
과천시,77775


In [165]:
park.head()

,공원면적
시군구,
가평군,511673
고양시덕양구,3287283
고양시일산동구,2722980
고양시일산서구,922486
과천시,315666


In [166]:
# 3. 1번과 2번 데이터프레임을 합성
park_pop = pd.merge(pop, park, left_index=True, right_index=True)
park_pop.head()

,총 인구수,공원면적
시군구,,
가평군,62197,511673
고양시덕양구,487874,3287283
고양시일산동구,296590,2722980
고양시일산서구,290738,922486
과천시,77775,315666


In [167]:
# 인당 공원면적 구하기
park_pop['인당 공원면적'] = (park_pop['공원면적'] / park_pop['총 인구수']).round(4)
park_pop.head()

,총 인구수,공원면적,인당 공원면적
시군구,,,
가평군,62197,511673,8.2267
고양시덕양구,487874,3287283,6.7380
고양시일산동구,296590,2722980,9.1810
고양시일산서구,290738,922486,3.1729
과천시,77775,315666,4.0587
